In [1]:
import numpy as np
import keras
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [3]:
def split_data(data):
        X = data[:,:-1]
        y = data[:,-1]
        X_train, X_test, y_train, y_test = train_test_split(X, 
                                                            y,
                                                            test_size=0.2,
                                                            random_state=42)
        return X_train, X_test, y_train, y_test
        

def create_data(number_of_data:int, low_value:int, high_value:int) -> list:
        """Helper function for creating data between the interval of low_value and high_value."""
        data = []
        for i in range(number_of_data):
            x1 = np.random.randint(low=low_value, high=high_value)
            x2 = np.random.randint(low=low_value, high=high_value)
            if x1 < high_value//2 and x2 > high_value//2:
                data.append([x1,x2,1])
            elif x1 < high_value//2 and x2 < high_value//2:
                data.append([x1,x2,0])
            elif x1 > high_value//2 and x2 > high_value//2:
                data.append([x1,x2,0])
            else:
                data.append([x1,x2,1])
        data = split_data(np.asarray(data))
        return data  

In [26]:
X_train, X_test, y_train, y_test = create_data(number_of_data=1000, low_value=0, high_value=100)

In [8]:
# model = Input -> Layer -> Layer -> Output
model = [] #list()

model.append("Input")
model.append("Layer1")
model.append("Layer2")
model.append("Output")

model

['Input', 'Layer1', 'Layer2', 'Output']

In [11]:
data[0].shape

(800, 2)

In [12]:
data[0][0]

array([ 4, 23])

In [31]:
model = keras.models.Sequential()

model.add(keras.layers.Input(shape=(2, ), name="Input Layer"))
model.add(keras.layers.Dense(2, activation="relu", name="Layer1"))
model.add(keras.layers.Dense(1, activation="sigmoid", name="Layer2"))

In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer1 (Dense)              (None, 2)                 6         
                                                                 
 Layer2 (Dense)              (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer=keras.optimizers.adam_v2.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [34]:
X_train[0:2]

array([[50, 60],
       [34, 31]])

In [35]:
y_train[:2]

array([1, 0])

In [36]:
model.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
800/800 [==============================] - 9s 9ms/step - loss: 0.6700 - accuracy: 0.5900
Epoch 2/10
800/800 [==============================] - 7s 8ms/step - loss: 0.6288 - accuracy: 0.6538
Epoch 3/10
800/800 [==============================] - 5s 6ms/step - loss: 0.6039 - accuracy: 0.6662
Epoch 4/10
800/800 [==============================] - 4s 5ms/step - loss: 0.5920 - accuracy: 0.6687
Epoch 5/10
800/800 [==============================] - 4s 5ms/step - loss: 0.5841 - accuracy: 0.6812
Epoch 6/10
800/800 [==============================] - 3s 4ms/step - loss: 0.5830 - accuracy: 0.6775
Epoch 7/10
800/800 [==============================] - 3s 4ms/step - loss: 0.5764 - accuracy: 0.6900
Epoch 8/10
800/800 [==============================] - 3s 4ms/step - loss: 0.5759 - accuracy: 0.6925
Epoch 9/10
800/800 [==============================] - 3s 4ms/step - loss: 0.5733 - accuracy: 0.6862
Epoch 10/10
800/800 [==============================] - 5s 7ms/step - loss: 0.5674 - accuracy: 0.7013

In [38]:
y_pred = 1 * (model.predict(X_test) > 0.5)
print("Confusion Matrix: \n {}".format(confusion_matrix(y_test, y_pred)))
print('Accuracy on Test Data:{:.3}'.format(accuracy_score(y_test, y_pred)))

Confusion Matrix: 
 [[83 19]
 [39 59]]
Accuracy on Test Data:0.71


## Pytorch

In [41]:
import torch
import torch.nn as nn

In [43]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.dense1 = nn.Linear(2, 2)
        self.dense2 = nn.Linear(2, 1)
        
    
    def forward(self, x):
        x_1 = torch.relu(self.dense1(x))
        x_2 = torch.sigmoid(self.dense2(x_1))
        return x_2
    
model = Net()

In [44]:
model

Net(
  (dense1): Linear(in_features=2, out_features=2, bias=True)
  (dense2): Linear(in_features=2, out_features=1, bias=True)
)

In [45]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

In [47]:
X_train_t = torch.from_numpy(X_train)
X_test_t = torch.from_numpy(X_test)
y_train_t = torch.from_numpy(y_train)
y_test_t = torch.from_numpy(y_test)

In [48]:
trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train_t, y_train_t), batch_size=1)

In [61]:
for i in range(10):
    for t, data in enumerate(trainloader):
        inputs, labels = data
        optimizer.zero_grad()
        y_pred = model(inputs.float()) # forward pass
        loss = criterion(y_pred, labels.float().reshape(-1,1)) # Error Calculation
        loss.backward() # Backpropagation
        optimizer.step() # Weight update

In [62]:
y_pred = model(X_test_t.float())
y_pred = y_pred.detach().numpy()

In [63]:
y_pred = 1 * (y_pred  > 0.5)
print("Confusion Matrix: \n {}".format(confusion_matrix(y_test_t, y_pred)))
print('Accuracy on Test Data:{:.3}'.format(accuracy_score(y_test_t, y_pred)))

Confusion Matrix: 
 [[102   0]
 [ 98   0]]
Accuracy on Test Data:0.51
